## 1. 실습1 - gensim 사용

### 1) 정수 인코딩과 단어 집합 만들기

- 뉴스그룹 데이터 불러오기

In [1]:
import pandas as pd
from sklearn.datasets import fetch_20newsgroups
dataset = fetch_20newsgroups(shuffle=True, random_state=1, remove=('headers', 'footers', 'quotes'))
documents = dataset.data

- 텍스트 데이터 전처리

In [2]:
news_df = pd.DataFrame({'document':documents})
# 특수 문자 제거
news_df['clean_doc'] = news_df['document'].str.replace("[^a-zA-Z]", " ")
# 길이가 3이하인 단어는 제거 (길이가 짧은 단어 제거)
news_df['clean_doc'] = news_df['clean_doc'].apply(lambda x: ' '.join([w for w in x.split() if len(w)>3]))
# 전체 단어에 대한 소문자 변환
news_df['clean_doc'] = news_df['clean_doc'].apply(lambda x: x.lower())

<ipython-input-2-b4124dfb5e6a>:3: FutureWarning: The default value of regex will change from True to False in a future version.
  news_df['clean_doc'] = news_df['document'].str.replace("[^a-zA-Z]", " ")


In [3]:
from nltk.corpus import stopwords
stop_words = stopwords.words('english') # NLTK로부터 불용어를 받아옴.
tokenized_doc = news_df['clean_doc'].apply(lambda x: x.split()) # 토큰화
tokenized_doc = tokenized_doc.apply(lambda x: [item for item in x if item not in stop_words])

In [4]:
tokenized_doc[:5]

0    [well, sure, story, seem, biased, disagree, st...
1    [yeah, expect, people, read, actually, accept,...
2    [although, realize, principle, strongest, poin...
3    [notwithstanding, legitimate, fuss, proposal, ...
4    [well, change, scoring, playoff, pool, unfortu...
Name: clean_doc, dtype: object

- 각 단어를 (word_id, word_frequency)의 형태로 바꾸고자 함.  
\- word_id : 단어가 정수 인코딩된 값  
\- word_frequency : 해당 뉴스에서의 해당 단어의 빈도수

In [5]:
from gensim import corpora
# 전체 뉴스에 대해서 정수 인코딩
dictionary = corpora.Dictionary(tokenized_doc)
corpus = [dictionary.doc2bow(text) for text in tokenized_doc]
print(corpus[1]) # 수행된 결과에서 두번째 뉴스 출력. 첫번째 문서의 인덱스는 0

C:\Users\colin\anaconda3\envs\DH\lib\site-packages\gensim\similarities\__init__.py:15: UserWarning: The gensim.similarities.levenshtein submodule is disabled, because the optional Levenshtein package <https://pypi.org/project/python-Levenshtein/> is unavailable. Install Levenhstein (e.g. `pip install python-Levenshtein`) to suppress this warning.
  warnings.warn(msg)


[(52, 1), (55, 1), (56, 1), (57, 1), (58, 1), (59, 1), (60, 1), (61, 1), (62, 1), (63, 1), (64, 1), (65, 1), (66, 2), (67, 1), (68, 1), (69, 1), (70, 1), (71, 2), (72, 1), (73, 1), (74, 1), (75, 1), (76, 1), (77, 1), (78, 2), (79, 1), (80, 1), (81, 1), (82, 1), (83, 1), (84, 1), (85, 2), (86, 1), (87, 1), (88, 1), (89, 1)]


> (66, 2)는 정수 인코딩이 66으로 할당된 단어가 두번째 뉴스에서는 두 번 등장하였음을 의미

In [6]:
print(dictionary[66])

faith


> 66이라는 값을 가지는 단어는 'faith'임.

In [7]:
len(dictionary)

64281

> 총 학습된 단어의 개수 : 64281개

### 2) LDA 모델 훈련시키기

In [8]:
import gensim
NUM_TOPICS = 20 #20개의 토픽, k=20
ldamodel = gensim.models.ldamodel.LdaModel(corpus, num_topics = NUM_TOPICS, id2word=dictionary, passes=15)
topics = ldamodel.print_topics(num_words=4)
for topic in topics:
    print(topic)

(0, '0.014*"information" + 0.013*"available" + 0.012*"data" + 0.010*"university"')
(1, '0.019*"file" + 0.012*"program" + 0.009*"files" + 0.008*"window"')
(2, '0.088*"space" + 0.025*"nasa" + 0.019*"launch" + 0.015*"satellite"')
(3, '0.017*"islam" + 0.016*"book" + 0.015*"remark" + 0.013*"theory"')
(4, '0.024*"insurance" + 0.015*"symbol" + 0.012*"germany" + 0.010*"france"')
(5, '0.021*"game" + 0.020*"team" + 0.015*"games" + 0.015*"play"')
(6, '0.026*"wire" + 0.026*"ground" + 0.015*"wiring" + 0.014*"electrical"')
(7, '0.017*"runs" + 0.015*"year" + 0.011*"lost" + 0.010*"ball"')
(8, '0.019*"hockey" + 0.010*"city" + 0.010*"sports" + 0.009*"roger"')
(9, '0.015*"would" + 0.012*"like" + 0.010*"think" + 0.010*"know"')
(10, '0.010*"would" + 0.009*"drive" + 0.009*"thanks" + 0.009*"system"')
(11, '0.009*"jesus" + 0.009*"would" + 0.008*"people" + 0.007*"believe"')
(12, '0.011*"government" + 0.008*"people" + 0.007*"armenian" + 0.006*"armenians"')
(13, '0.044*"image" + 0.021*"images" + 0.018*"graphics"

> 각 단어 앞에 붙은 수치는 단어의 해당 토픽에 대한 기여도를 보여줌.  
> passes : 알고리즘의 동작 횟수. 토픽의 값이 적절히 수렴할 수 있도록 충분한 횟수를 정해줌.  
> num_words=n : 총 n개의 단어만 출력. (default = 10)

### 3) LDA 시각화 하기

In [9]:
#!pip install pyLDAvis
import pyLDAvis
import pyLDAvis.gensim_models as gensimvis

pyLDAvis.enable_notebook()
vis = gensimvis.prepare(ldamodel, corpus, dictionary)
pyLDAvis.display(vis)

> 좌측의 원들은 각각의 20개의 토픽을 나타냄.  
> 각 원과의 거리는 각 토픽들이 서로 얼마나 다른지를 보여줌.  
> 즉, 만약 두 개의 원이 겹친다면, 이 두 개의 토픽은 유사한 토픽이라는 의미.  
> 위의 그림에서는 10번 토픽을 클릭하였고, 이에 따라 우측에는 10번 토픽에 대한 정보가 나타남.  
> LDA 시각화에서는 토픽의 번호가 1부터 시작하므로 각 토픽 번호는 이제 +1이 된 값인 1~20까지의 값을 가짐.

### 4) 문서 별 토픽 분포 보기

In [10]:
for i, topic_list in enumerate(ldamodel[corpus]):
    if i==5:  # 상위 5개의 문서에 대해서만 토픽 분포를 확인
        break
    print(i,'번째 문서의 topic 비율은',topic_list)

0 번째 문서의 topic 비율은 [(8, 0.028220577), (9, 0.50075346), (12, 0.17686784), (14, 0.2810203)]
1 번째 문서의 topic 비율은 [(7, 0.027157798), (9, 0.32804304), (11, 0.59613645), (18, 0.0275897)]
2 번째 문서의 topic 비율은 [(9, 0.303776), (11, 0.14587416), (14, 0.5365926)]
3 번째 문서의 topic 비율은 [(9, 0.42864895), (10, 0.123212196), (12, 0.3520439), (16, 0.053966932), (17, 0.016628599), (18, 0.014518457)]
4 번째 문서의 topic 비율은 [(1, 0.06814122), (2, 0.20865035), (5, 0.32057044), (9, 0.37299913)]


C:\Users\colin\anaconda3\envs\DH\lib\site-packages\ipykernel\ipkernel.py:287: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


> (숫자, 확률) : 각각 토픽 번호와 해당 토픽이 해당 문서에서 차지하는 분포도  
> (3, 0.045884132) : 7번 토픽이 약 4.59%의 분포도를 가지는 것을 의미.

In [11]:
# 깔끔한 형태인 데이터프레임 형식으로 출력 함수
def make_topictable_per_doc(ldamodel, corpus):
    topic_table = pd.DataFrame()

    # 몇 번째 문서인지를 의미하는 문서 번호와 해당 문서의 토픽 비중을 한 줄씩 꺼내옴.
    for i, topic_list in enumerate(ldamodel[corpus]):
        doc = topic_list[0] if ldamodel.per_word_topics else topic_list            
        doc = sorted(doc, key=lambda x: (x[1]), reverse=True)
        # 각 문서에 대해서 비중이 높은 토픽순으로 토픽을 정렬.
        # EX) 정렬 전 0번 문서 : (2번 토픽, 48.5%), (8번 토픽, 25%), (10번 토픽, 5%), (12번 토픽, 21.5%), 
        # Ex) 정렬 후 0번 문서 : (2번 토픽, 48.5%), (8번 토픽, 25%), (12번 토픽, 21.5%), (10번 토픽, 5%)
        # 48 > 25 > 21 > 5 순으로 정렬이 된 것.

        # 모든 문서에 대해서 각각 아래를 수행
        for j, (topic_num, prop_topic) in enumerate(doc): #  몇 번 토픽인지와 비중을 나눠서 저장.
            if j == 0:  # 정렬을 한 상태이므로 가장 앞에 있는 것이 가장 비중이 높은 토픽
                topic_table = topic_table.append(pd.Series([int(topic_num), round(prop_topic,4), topic_list]), ignore_index=True)
                # 가장 비중이 높은 토픽과, 가장 비중이 높은 토픽의 비중과, 전체 토픽의 비중을 저장.
            else:
                break
    return(topic_table)

C:\Users\colin\anaconda3\envs\DH\lib\site-packages\ipykernel\ipkernel.py:287: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


In [12]:
topictable = make_topictable_per_doc(ldamodel, corpus)
topictable = topictable.reset_index() # 문서 번호을 의미하는 열(column)로 사용하기 위해서 인덱스 열을 하나 더 만듦.
topictable.columns = ['문서 번호', '가장 비중이 높은 토픽', '가장 높은 토픽의 비중', '각 토픽의 비중']
topictable[:10]

C:\Users\colin\anaconda3\envs\DH\lib\site-packages\ipykernel\ipkernel.py:287: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


,문서 번호,가장 비중이 높은 토픽,가장 높은 토픽의 비중,각 토픽의 비중
0,0,9.0,0.5008,"[(8, 0.028218502), (9, 0.5007528), (12, 0.1768..."
1,1,11.0,0.5962,"[(7, 0.027158072), (9, 0.3279989), (11, 0.5961..."
2,2,14.0,0.5366,"[(9, 0.30401802), (11, 0.14561947), (14, 0.536..."
3,3,9.0,0.4287,"[(9, 0.42870927), (10, 0.12313998), (12, 0.352..."
4,4,9.0,0.3730,"[(1, 0.068198204), (2, 0.20864098), (5, 0.3205..."
5,5,11.0,0.4176,"[(9, 0.3720263), (11, 0.4175943), (12, 0.05493..."
6,6,10.0,0.4724,"[(0, 0.041169383), (1, 0.0842644), (7, 0.03874..."
7,7,9.0,0.4936,"[(9, 0.4935685), (12, 0.18599443), (14, 0.3064..."
8,8,9.0,0.6178,"[(1, 0.13410424), (3, 0.029848594), (9, 0.6177..."
9,9,9.0,0.5243,"[(9, 0.5242529), (10, 0.276578), (12, 0.147285..."


## 2. 실습2 - sklearn 사용

### 1) 뉴스 기사 제목 데이터에 대한 이해

In [13]:
import pandas as pd
import urllib.request
urllib.request.urlretrieve("https://raw.githubusercontent.com/franciscadias/data/master/abcnews-date-text.csv", filename="abcnews-date-text.csv")
# 약 15년 동안 발행되었던 뉴스 기사 제목을 모아놓은 영어 데이터
data = pd.read_csv('../Data/abcnews-date-text.csv', error_bad_lines=False)
print(len(data))

C:\Users\colin\anaconda3\envs\DH\lib\site-packages\ipykernel\ipkernel.py:287: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


1226258


> 해당 데이터는 약 122만개의 샘플을 갖고 있음.

In [14]:
# 상위 5개의 샘플
print(data.head(5))

   publish_date                                      headline_text
0      20030219  aba decides against community broadcasting lic...
1      20030219     act fire witnesses must be aware of defamation
2      20030219     a g calls for infrastructure protection summit
3      20030219           air nz staff in aust strike for pay rise
4      20030219      air nz strike to affect australian travellers


C:\Users\colin\anaconda3\envs\DH\lib\site-packages\ipykernel\ipkernel.py:287: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


In [15]:
# 뉴스 기사 제목
text = data[['headline_text']]
text.head(5)

C:\Users\colin\anaconda3\envs\DH\lib\site-packages\ipykernel\ipkernel.py:287: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


,headline_text
0,aba decides against community broadcasting lic...
1,act fire witnesses must be aware of defamation
2,a g calls for infrastructure protection summit
3,air nz staff in aust strike for pay rise
4,air nz strike to affect australian travellers


### 2) 텍스트 전처리

In [16]:
import nltk
# NLTK의 word_tokenize를 통해 단어 토큰화를 수행
text['headline_text'] = text.apply(lambda row: nltk.word_tokenize(row['headline_text']), axis=1)
print(text.head(5))

C:\Users\colin\anaconda3\envs\DH\lib\site-packages\ipykernel\ipkernel.py:287: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


                                       headline_text
0  [aba, decides, against, community, broadcastin...
1  [act, fire, witnesses, must, be, aware, of, de...
2  [a, g, calls, for, infrastructure, protection,...
3  [air, nz, staff, in, aust, strike, for, pay, r...
4  [air, nz, strike, to, affect, australian, trav...


<ipython-input-16-844c6e452e11>:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  text['headline_text'] = text.apply(lambda row: nltk.word_tokenize(row['headline_text']), axis=1)


In [17]:
# 불용어를 제거
from nltk.corpus import stopwords
stop = stopwords.words('english')
text['headline_text'] = text['headline_text'].apply(lambda x: [word for word in x if word not in (stop)])
print(text.head(5))

C:\Users\colin\anaconda3\envs\DH\lib\site-packages\ipykernel\ipkernel.py:287: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


                                       headline_text
0   [aba, decides, community, broadcasting, licence]
1    [act, fire, witnesses, must, aware, defamation]
2     [g, calls, infrastructure, protection, summit]
3          [air, nz, staff, aust, strike, pay, rise]
4  [air, nz, strike, affect, australian, travellers]


<ipython-input-17-b8e1d573c30a>:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  text['headline_text'] = text['headline_text'].apply(lambda x: [word for word in x if word not in (stop)])


> against, be, of, a, in, to 등의 단어가 제거됨.

In [18]:
#nltk.download('wordnet')
# 표제어 추출 :  3인칭 단수 표현을 1인칭으로 바꾸고, 과거 현재형 동사를 현재형으로 바꿈.
from nltk.stem import WordNetLemmatizer
text['headline_text'] = text['headline_text'].apply(lambda x: [WordNetLemmatizer().lemmatize(word, pos='v') for word in x])
print(text.head(5))

C:\Users\colin\anaconda3\envs\DH\lib\site-packages\ipykernel\ipkernel.py:287: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


                                       headline_text
0       [aba, decide, community, broadcast, licence]
1      [act, fire, witness, must, aware, defamation]
2      [g, call, infrastructure, protection, summit]
3          [air, nz, staff, aust, strike, pay, rise]
4  [air, nz, strike, affect, australian, travellers]


<ipython-input-18-c88ff4091630>:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  text['headline_text'] = text['headline_text'].apply(lambda x: [WordNetLemmatizer().lemmatize(word, pos='v') for word in x])


In [19]:
# 길이가 3이하인 단어에 대해서 제거
tokenized_doc = text['headline_text'].apply(lambda x: [word for word in x if len(word) > 3])
print(tokenized_doc[:5])

C:\Users\colin\anaconda3\envs\DH\lib\site-packages\ipykernel\ipkernel.py:287: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


0       [decide, community, broadcast, licence]
1      [fire, witness, must, aware, defamation]
2    [call, infrastructure, protection, summit]
3                   [staff, aust, strike, rise]
4      [strike, affect, australian, travellers]
Name: headline_text, dtype: object


### 3) TF-IDF 행렬 만들기

In [20]:
# TfidfVectorizer는 기본적으로 토큰화가 되어있지 않은 텍스트 데이터를 입력으로 사용.
# 역토큰화 (토큰화 작업을 되돌림)
detokenized_doc = []
for i in range(len(text)):
    t = ' '.join(tokenized_doc[i])
    detokenized_doc.append(t)

text['headline_text'] = detokenized_doc # 다시 text['headline_text']에 재저장
text['headline_text'][:5] # 5개의 샘플

C:\Users\colin\anaconda3\envs\DH\lib\site-packages\ipykernel\ipkernel.py:287: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)
<ipython-input-20-95179c13e001>:8: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  text['headline_text'] = detokenized_doc # 다시 text['headline_text']에 재저장


0       decide community broadcast licence
1       fire witness must aware defamation
2    call infrastructure protection summit
3                   staff aust strike rise
4      strike affect australian travellers
Name: headline_text, dtype: object

In [21]:
from sklearn.feature_extraction.text import TfidfVectorizer
vectorizer = TfidfVectorizer(stop_words='english', 
max_features= 1000) # 상위 1,000개의 단어를 보존 
X = vectorizer.fit_transform(text['headline_text'])
X.shape # TF-IDF 행렬의 크기 확인

C:\Users\colin\anaconda3\envs\DH\lib\site-packages\ipykernel\ipkernel.py:287: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


(1226258, 1000)

> 텍스트 데이터에 있는 모든 단어를 가지고 TF-IDF 행렬을 만들 수도 있겠지만, 여기서는 간단히 1,000개의 단어로 제한  
> 1,226,258 × 1,000의 크기를 가진 가진 TF-IDF 행렬이 생김.

### 4) 토픽 모델링(LDA 수행)

In [22]:
from sklearn.decomposition import LatentDirichletAllocation
lda_model=LatentDirichletAllocation(n_components=10,learning_method='online',random_state=777,max_iter=1)
lda_top=lda_model.fit_transform(X)
print(lda_model.components_)
print()
print(lda_model.components_.shape) 

C:\Users\colin\anaconda3\envs\DH\lib\site-packages\ipykernel\ipkernel.py:287: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


[[1.00000865e-01 1.00000439e-01 1.00001997e-01 ... 1.00006873e-01
  1.00003405e-01 1.00005209e-01]
 [1.00001642e-01 1.00000829e-01 6.40533260e+02 ... 1.00009111e-01
  1.00004890e-01 5.79474578e+02]
 [1.00001468e-01 1.00000275e-01 1.00001496e-01 ... 1.00004592e-01
  1.00001786e-01 1.00005396e-01]
 ...
 [1.00002822e-01 1.00000923e-01 1.00001462e-01 ... 1.00009141e-01
  1.00005015e-01 1.00008595e-01]
 [1.00004695e-01 1.00002038e-01 1.00001419e-01 ... 1.00004812e-01
  1.00002548e-01 1.00007925e-01]
 [1.07423402e+02 2.03964360e+02 1.00002187e-01 ... 1.00006822e-01
  1.00003052e-01 1.00006310e-01]]

(10, 1000)


In [23]:
terms = vectorizer.get_feature_names() # 단어 집합. 1,000개의 단어가 저장됨.

def get_topics(components, feature_names, n=5):
    for idx, topic in enumerate(components):
        print("Topic %d:" % (idx+1), [(feature_names[i], topic[i].round(2)) for i in topic.argsort()[:-n - 1:-1]])

get_topics(lda_model.components_,terms)

Topic 1: [('queensland', 12908.75), ('sydney', 10948.96), ('melbourne', 8900.08), ('change', 7262.83), ('crash', 6153.23)]
Topic 2: [('australia', 19355.9), ('australian', 13286.11), ('leave', 4930.96), ('speak', 4845.83), ('perth', 4709.2)]
Topic 3: [('donald', 9114.15), ('live', 7908.02), ('federal', 4711.88), ('rise', 4630.85), ('victorian', 4567.89)]
Topic 4: [('health', 6349.63), ('tasmania', 6141.91), ('report', 5567.09), ('plan', 4834.44), ('time', 4744.41)]
Topic 5: [('state', 6086.17), ('open', 6074.01), ('coast', 6015.08), ('restrictions', 5961.35), ('woman', 5921.79)]
Topic 6: [('trump', 15903.94), ('police', 13931.38), ('home', 7318.39), ('test', 7241.0), ('market', 6529.05)]
Topic 7: [('government', 9187.9), ('record', 6384.64), ('border', 6378.89), ('help', 5807.23), ('people', 5620.56)]
Topic 8: [('coronavirus', 48038.98), ('covid', 19540.99), ('victoria', 10827.28), ('china', 8358.46), ('death', 7181.89)]
Topic 9: [('case', 10138.89), ('charge', 8386.79), ('court', 8195

C:\Users\colin\anaconda3\envs\DH\lib\site-packages\ipykernel\ipkernel.py:287: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)
